In [1]:
from datetime import date
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
import os
import re
import time
import random

# Список ссылок по дням

In [17]:
BEGIN_LENTA = 730120

In [12]:
DELTA_LENTA = int(re.search('[0-9]+', str(date.today() - date.fromordinal(BEGIN_LENTA))).group(0))

In [19]:
def overall_links_lenta():
    date_l = []
    for i in range(BEGIN_LENTA, BEGIN_LENTA+DELTA_LENTA):
        d = date.fromordinal(i)
        dd = d.strftime("https://lenta.ru/news/20%y/%m/%d/")
        date_l.append(dd)
    return date_l

In [20]:
date_l = overall_links_lenta()
print(len(date_l))

7453


In [6]:
date_l[1825]

'https://lenta.ru/news/2004/12/30/'

## Формирование ссылок

In [21]:
def lenta_year_links(year, date_l):
    links = []
    if year != 2000:
        begin = 365*(year-2000)
    else:
        begin = 0
    for i in date_l[begin:begin+367]:
        if i.startswith('https://lenta.ru/news/{}'.format(year)):
            res = requests.get(i)
            soup = BeautifulSoup(res.content, 'lxml')
            for t in soup.find_all('div', {'class': 'titles'}):
                link = 'https://lenta.ru' + t.h3.a['href']
                links.append(link)
    return links

In [22]:
l2001 = lenta_year_links(2001, date_l)

In [23]:
len(l2001)

22144

# Сбор текста из статьи

In [26]:
os.getcwd()

'D:\\Downloads\\Documents\\Lenta.ru\\lenta2001'

In [25]:
os.chdir('../lenta2001')

In [27]:
def lenta_downloader(links_list, corp_dir, begin, end):
    if os.path.exists(corp_dir) and os.path.isdir(corp_dir):
        os.chdir(corp_dir)
    else:
        os.mkdir(corp_dir)
        os.chdir(corp_dir)
    for link in links_list[begin:end]:
        link1 = re.sub(r'/', '', link[22:-1])
        if os.path.exists(str(link1 + '.txt')):
            pass
        else:
            try:
                req = requests.get(link)
                soup = BeautifulSoup(req.content, 'lxml')
                title = soup.find('h1', {'class': 'b-topic__title'}).text
                time = soup.find('time', {'class': 'g-date'})['datetime']
                txt = soup.find('div', {'class': 'b-text clearfix js-topic__text'}).text
                with open('lenta{}.txt'.format(link1), 'w', encoding="UTF-8") as f:
                    f.write(time + '\n' + title + '\n' + txt)
            except (AttributeError, TypeError) as e:
                pass

In [33]:
lenta_downloader(l2001, 'D:/Downloads/Documents/Lenta.ru/lenta2001', 10000, 22145)

На случай обрыва соединения

In [32]:
for i in range(0,15000):
    if l2001[i] == 'https://lenta.ru/news/2008/06/18/fist':
        print(i)

## Для скачивания html

In [50]:
for link in links2000[7501:8000]:
    req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read().decode("utf-8")
    link1 = re.sub(r'/', '', link[22:-1])
    randsleep = random.randint(3,9)/10
    time.sleep (randsleep)
    with open('{}.html'.format(link1), "w", encoding="UTF-8") as f:
        f.write(page)

In [8]:
os.chdir('lenta2001')

с 7501 до 8000 - начинаются txt
с 10531 - без деления на абзацы